In [1]:
import numpy as np
import pandas as pd
import utm
import math
import pickle
import os
import sys
sys.path.append('./src')
import time
import datetime
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import copy
import tensorflow as tf
from importlib import reload

import load, plot
import tools
import pygrid
import process

D:\Programs\Anaconda2\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load Data

In [2]:
dataset = 'jiading'
datatype = '2g'
data_file = './data/%s_%s/data_%s.csv' % (dataset, datatype, datatype)
gongcan_file = './data/%s_%s/gongcan_%s.csv' % (dataset, datatype, datatype)
disp_path = './display/%s_%s/' % (dataset, datatype)
data_path = './data/%s_%s/' % (dataset, datatype)
map_file = './data/%s_map/%s_EdgeGeometry.txt' % (dataset, dataset)
edge_file = './data/%s_map/%s_Edges.txt' % (dataset, dataset)
node_file = './data/%s_map/%s_Nodes.txt' % (dataset, dataset)

In [3]:
load = reload(load)
tools = reload(tools)
df_raw = load.load_data(data_file, gongcan_file)
max_dist, min_dist, min_len, max_again = load.get_config(dataset, datatype)
df = load.clean_data(df_raw, max_dist, min_dist, min_len, max_again, debug=True)

Traj ID=0
0->1:286m	
discard[0:0]=0m	keep[1:24]as id=0	
[24]
Traj ID=1
32->33:510m	
discard[25:32]=0m	keep[33:182]as id=1	
[150]
Traj ID=2

keep[183:216]as id=2	
[34]
Traj ID=3
217->218:394m	
discard[217:217]=0m	keep[218:261]as id=3	
[44]
Traj ID=4
263->264:118m	
discard[262:263]=0m	keep[264:311]as id=4	
[48]
Traj ID=5
313->314:522m	424->425:752m	445->446:116m	
discard[312:313]=0m	trimed[0,89] keep[314:335]as id=5	discard[425:445]=33m	discard[446:447]=0m	
[22]
Traj ID=6
457->458:101m	
discard[448:457]=0m	keep[458:560]as id=6	
[103]
Traj ID=7

keep[561:934]as id=7	
[374]
Traj ID=8
936->937:947m	
discard[935:936]=0m	keep[937:1276]as id=8	
[340]
Traj ID=9

keep[1277:1697]as id=9	
[421]
Traj ID=10

keep[1698:1993]as id=10	
[296]
Traj ID=11

keep[1994:2416]as id=11	
[423]
Traj ID=12
2452->2453:838m	
discard[2417:2452]=0m	keep[2453:3432]as id=12	
[980]
Traj ID=13

keep[3433:4259]as id=13	
[827]
Traj ID=14
4302->4303:667m	
keep[4260:4302]as id=14	keep[4303:4374]as id=15	
[43, 72]
Traj ID=15
4

In [5]:
# df.to_csv(data_path + 'data_%s_clean.csv' % datatype, index=False)
df = pd.read_csv(data_path + 'data_%s_clean.csv' % datatype)

In [6]:
print (len(df_raw), len(df))

13191 12517


In [353]:
load = reload(load)
load.to_trajs(df, data_path + 'trajs')

In [355]:
display = reload(display)
max_trid = max(df['TrajID'])
for tr_id in range(max_trid + 1):
    df_i = df[df['TrajID']==tr_id]
    display.df_to_html(df_i, disp_path + 'path/%d.html' % tr_id, ('Latitude', 'Longitude'), more_info=False)

# Map Matching

In [7]:
load = reload(load)
useful_col = ['TrajID', 'MRTime', 'RoadID', 'Match_Lat', 'Match_Lng', 'Match_loc', 'Match_Dist']
match_df = load.load_matching(data_path + 'matching_out', max(df['TrajID']), 50)[useful_col]
len(match_df)

12388

In [8]:
df = pd.merge(df, match_df, on=['TrajID', 'MRTime'])
df = load.fill_utm_axis(df, ['Match_Lat', 'Match_Lng'], ['Match_UTM_X', 'Match_UTM_Y'])
len(df)

12388

In [373]:
max_trid = max(df['TrajID'])
for tr_id in range(max_trid + 1):
    df_i = df[df['TrajID']==tr_id]
    display.df_to_html(df_i, disp_path + 'match/%d.html' % tr_id, ('Match_Lat', 'Match_Lng'), more_info=True)

In [9]:
load = reload(load)
roadmap, edgemap, nodemap = load.load_map(map_file, edge_file, node_file, dataset)
print (len(roadmap), len(edgemap), len(nodemap))

550 550 210


In [321]:
display = reload(display)
display.map_to_html(roadmap, './display/%s_map.html' % dataset)

# Find Patterns

In [10]:
pygrid = reload(pygrid)
if dataset == 'jiading':
    bounding_box = (328500, 330600, 3462100, 3463600)
elif dataset == 'siping':
    bounding_box = (356000, 359000, 3461500, 3463000)
side = 20
grid = pygrid.Grid(side, bounding_box)

In [57]:
process = reload(process)
load = reload(load)
towers, r_towers = load.load_gongcan(gongcan_file)
w_size = 5
cellids, patterns, statistic = process.retrive(df, grid, towers, edgemap, nodemap, w_size)
print (len(cellids), len(patterns))
print (patterns[0])

11601 11601
[(329964.3817175407, 3463039.681154026), (329960.64643888554, 3463039.2014535726), (329961.7555243264, 3463039.343893213), (329964.11835392227, 3463039.647331306), (329963.26938791166, 3463039.5383009496)]


In [64]:
process = reload(process)
labels, label_set, discard_idxs, len_stat = process.pattern2label(patterns, grid, version=2)
features = process.cellids2feature(cellids, r_towers, discard_idxs, version=0)

Origin Labels: 10825
Merged labels: 28


In [65]:
features[0]

[80,
 31.285957,
 121.21734099999999,
 -79,
 88,
 31.291315,
 121.217252,
 -83,
 53,
 31.287758,
 121.22313100000001,
 -83,
 79,
 31.286457000000002,
 121.215515,
 -81,
 85,
 31.28055,
 121.213089,
 -91,
 84,
 31.28055,
 121.213089,
 -87,
 81,
 31.285957,
 121.21734099999999,
 -77,
 81,
 31.285957,
 121.21734099999999,
 -75,
 88,
 31.291315,
 121.217252,
 -81,
 53,
 31.287758,
 121.22313100000001,
 -83,
 79,
 31.286457000000002,
 121.215515,
 -83,
 85,
 31.28055,
 121.213089,
 -91,
 84,
 31.28055,
 121.213089,
 -87,
 -1,
 0,
 0,
 -1,
 81,
 31.285957,
 121.21734099999999,
 -75,
 88,
 31.291315,
 121.217252,
 -81,
 53,
 31.287758,
 121.22313100000001,
 -83,
 79,
 31.286457000000002,
 121.215515,
 -83,
 85,
 31.28055,
 121.213089,
 -91,
 84,
 31.28055,
 121.213089,
 -87,
 80,
 31.285957,
 121.21734099999999,
 -79,
 81,
 31.285957,
 121.21734099999999,
 -73,
 88,
 31.291315,
 121.217252,
 -83,
 53,
 31.287758,
 121.22313100000001,
 -81,
 79,
 31.286457000000002,
 121.215515,
 -81,
 85,
 31

In [441]:
len([t for t in len_stat if t > 60]) / len(len_stat)

0.09818181818181818

In [13]:
summary = dict()
for label in labels:
    pattern = label_set[label]
    if pattern not in summary.keys():
        summary[pattern] = 0
    summary[pattern] += 1
total = sum(cnt for cnt in summary.values())
# print ((summary[(0,)] + summary[(1,)] + summary[(3,)] + summary[(5,)] + summary[(7,)]) / total)
more2 = 0
for pat, cnt in summary.items():
    if len(pat) > 2:
        more2 += cnt
    print (pat, ':', cnt, '%.3f' % (cnt / 1.0 / total))
print (more2 / 1.0 / total)

(0,) : 1647 0.152
(3,) : 2555 0.236
(1,) : 2070 0.191
(5,) : 1933 0.179
(1, 3) : 183 0.017
(7, 5) : 160 0.015
(7,) : 1591 0.147
(7, 1) : 104 0.010
(7, 1, 7) : 9 0.001
(1, 7) : 112 0.010
(7, 5, 7) : 10 0.001
(5, 7) : 136 0.013
(3, 1) : 115 0.011
(3, 1, 3) : 7 0.001
(1, 7, 1) : 4 0.000
(5, 3) : 68 0.006
(5, 7, 5) : 5 0.000
(5, 3, 5) : 6 0.001
(5, 3, 5, 3) : 1 0.000
(3, 5, 3) : 12 0.001
(5, 7, 5, 3) : 1 0.000
(3, 5) : 80 0.007
(7, 1, 3) : 2 0.000
(5, 7, 1) : 8 0.001
(7, 5, 3) : 1 0.000
(5, 3, 1) : 1 0.000
(1, 3, 1) : 2 0.000
(3, 5, 7) : 2 0.000
0.006558891454965358


In [ ]:
plot = reload(plot)
fig_width = 5
axis = (-3, 3, -3, 3)
for idx, pattern in enumerate(label_set):
    plt.figure(figsize=(fig_width, fig_width*(axis[3]-axis[2])/(axis[1]-axis[0])))
    ca = plt.gca()
    plot.draw_pattern(pattern, ca, axis, 'k', debug=False)
    plt.axis(axis)
    plt.title('LabelID=' + str(idx))
#     plt.show()
    plt.savefig(disp_path + 'patterns/LabelID=%d.png' % idx)

In [37]:
bounding_box

(328500, 330600, 3462100, 3463600)

In [15]:
def round_axis(coors, bounding, ct):
    x0, x1, y0, y1 = bounding
    xs, ys = [coor[0] for coor in coors], [coor[1] for coor in coors]
    mx, my = np.mean(xs), np.mean(ys)
    x0_, x1_, y0_, y1_ = int(mx - ct), int(mx + ct), int(my - ct), int(my + ct)
    return max(x0_, x0),  min(x1_, x1), max(y0_, y0), min(y1_, y1)

plot = reload(plot)
fig_width = 20
cnt = 0
for idx, pattern in enumerate(patterns):
    if idx in discard_idxs:
        continue
    label_id = labels[cnt]
    cnt += 1
    if len(label_set[label_id]) < 3:
        continue
    axis = round_axis(pattern, bounding_box, 300)
#     print (axis)
    plt.figure(figsize=(fig_width, fig_width*(axis[3]-axis[2])/(axis[1]-axis[0])))
    ca = plt.gca()
    plot.draw_map(roadmap, ca, axis)
    plot.draw_window_traj(pattern, ca, axis, 'k', debug=True)
    plt.axis(axis)
    plt.title('WindowID=%d_LabelID=%d'%(idx, label_id))
#     plt.show()
    plt.savefig(disp_path + 'pattern-traj/LabelID=%d_WindowID=%d.png' % (label_id, idx))
    plt.close()
    

# Split Data

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.25)

In [67]:
len(X_train), len(X_test)

(8118, 2707)

# Try RandomForest

In [70]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier(n_estimators=200)
# model = RandomForestRegressor(n_estimators=200)
model.fit(X_train, Y_train)
Y_predict = model.predict(X_test)
tools = reload(tools)
score = tools.accuracy(Y_test, Y_predict.round())
score

0.34096786110084965

In [43]:
model.feature_importances_

array([0.02979616, 0.03031118, 0.03343013, 0.03209802, 0.03319915,
       0.03001348, 0.0165716 , 0.02850816, 0.02862437, 0.03472881,
       0.03015807, 0.03055793, 0.02576935, 0.01490004, 0.02835399,
       0.02704854, 0.03446568, 0.03023946, 0.0310846 , 0.02444784,
       0.01376934, 0.02751484, 0.02952604, 0.03628921, 0.03146772,
       0.03135697, 0.02654919, 0.01409116, 0.03042616, 0.03074543,
       0.03582951, 0.0359151 , 0.0354411 , 0.03030579, 0.01646587])

# CNN Classification

In [413]:
from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_tr = X_train.reshape(-1, w_size, num_cellid, 1)
X_test = preprocessing.scale(X_test)
X_te = X_test.reshape(-1, w_size, num_cellid, 1)

In [416]:
num_class = len(label_set)
num_cellid = 7

In [417]:
from keras.utils.np_utils import to_categorical
Y_tr = to_categorical(pd.DataFrame(Y_train), num_classes=num_class)
Y_te = to_categorical(pd.DataFrame(Y_test), num_classes=num_class)

In [418]:
Y_tr[0]

array([0., 1.], dtype=float32)

In [419]:
Y_te[0]

array([0., 1.], dtype=float32)

In [420]:
num_cellid, w_size, num_class

(7, 5, 2)

## Basic Version

In [421]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

tf.reset_default_graph()
# Input Layer
tf_X = tf.placeholder(tf.float32, [None, w_size, num_cellid, 1])
tf_Y = tf.placeholder(tf.float32, [None, num_class])

# Conv Layer
# 卷积核太愚蠢
conv_w1 = tf.Variable(tf.random_normal([w_size, num_cellid, 1, num_class]))
conv_b1 = tf.Variable(tf.random_normal([num_class]))
relu_feature_map1 = tf.nn.relu(tf.nn.conv2d(tf_X, conv_w1, strides=[1,1,1,1], padding='SAME') + conv_b1)
print (relu_feature_map1)

# Pooling Layer
# max_pool1 = tf.nn.max_pool(relu_feature_map1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

# Conv Layer 2
conv_w2 = tf.Variable(tf.random_normal([w_size, num_cellid, num_class, 5]))
conv_b2 = tf.Variable(tf.random_normal([5]))
conv_out2 = tf.nn.conv2d(relu_feature_map1, conv_w2, strides=[1,2,2,1], padding='SAME') + conv_b2
print (conv_out2)

# Normalization
batch_mean, batch_var = tf.nn.moments(conv_out2, [0, 1, 2], keep_dims=True)
shift = tf.Variable(tf.zeros([5]))
scale = tf.Variable(tf.ones([5]))
epsilon = 1e-3
BN_out = tf.nn.batch_normalization(conv_out2, batch_mean, batch_var, shift, scale, epsilon)
print (BN_out)
relu_BN_map2 = tf.nn.relu(BN_out)

relu_BN_map2_flat = tf.reshape(relu_BN_map2, [-1, 3*4*5])

#Fully Connection
fc_w1 = tf.Variable(tf.random_normal([3*4*5, 64]))
fc_b1 = tf.Variable(tf.random_normal([64]))
fc_out1 = tf.nn.relu(tf.matmul(relu_BN_map2_flat, fc_w1) + fc_b1)

# Output Layer
out_w1 = tf.Variable(tf.random_normal([64, num_class]))
out_b1 = tf.Variable(tf.random_normal([num_class]))
pred = tf.nn.softmax(tf.matmul(fc_out1, out_w1) + out_b1)

loss = -tf.reduce_mean(tf_Y*tf.log(tf.clip_by_value(pred, 1e-11, 1.0)))
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
y_pred = tf.arg_max(pred, 1)
bool_pred = tf.equal(tf.arg_max(tf_Y, 1), y_pred)
accuracy = tf.reduce_mean(tf.cast(bool_pred, tf.float32))

Tensor("Relu:0", shape=(?, 5, 7, 2), dtype=float32)
Tensor("add_1:0", shape=(?, 3, 4, 5), dtype=float32)
Tensor("batchnorm/add_1:0", shape=(?, 3, 4, 5), dtype=float32)


## Experimental step

In [255]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

tf.reset_default_graph()
# Input Layer
tf_X = tf.placeholder(tf.float32, [None, w_size, num_cellid, 1])
tf_Y = tf.placeholder(tf.float32, [None, num_class])

# Conv Layer
# 卷积核太愚蠢
conv_w1 = tf.Variable(tf.random_normal([3, 3, 1, 32]))
conv_b1 = tf.Variable(tf.random_normal([32]))
conv_out1 = tf.nn.relu(tf.nn.conv2d(tf_X, conv_w1, strides=[1,1,1,1], padding='SAME') + conv_b1)
print (conv_out1)

# Conv Layer 2
conv_w2 = tf.Variable(tf.random_normal([3, 3, 32, 64]))
conv_b2 = tf.Variable(tf.random_normal([64]))
conv_out2 = tf.nn.relu(tf.nn.conv2d(conv_out1, conv_w2, strides=[1,1,1,1], padding='SAME') + conv_b2)
print (conv_out2)

# Conv Layer 3
conv_w3 = tf.Variable(tf.random_normal([3, 3, 64, 128]))
conv_b3 = tf.Variable(tf.random_normal([128]))
conv_out3 = tf.nn.relu(tf.nn.conv2d(conv_out2, conv_w3, strides=[1,1,1,1], padding='SAME') + conv_b3)
print (conv_out3)

# Normalization
batch_mean, batch_var = tf.nn.moments(conv_out3, [0, 1, 2], keep_dims=True)
shift = tf.Variable(tf.zeros([128]))
scale = tf.Variable(tf.ones([128]))
epsilon = 1e-3
BN_out = tf.nn.batch_normalization(conv_out3, batch_mean, batch_var, shift, scale, epsilon)
print (BN_out)
relu_BN_map2 = tf.nn.relu(BN_out)
relu_BN_map2_flat = tf.reshape(relu_BN_map2, [-1, w_size * num_cellid * 128])

#Fully Connection
fc_w1 = tf.Variable(tf.random_normal([w_size * num_cellid * 128, 1000]))
fc_b1 = tf.Variable(tf.random_normal([1000]))
fc_out1 = tf.nn.relu(tf.matmul(relu_BN_map2_flat, fc_w1) + fc_b1)

# Output Layer
out_w1 = tf.Variable(tf.random_normal([1000, num_class]))
out_b1 = tf.Variable(tf.random_normal([num_class]))
pred = tf.nn.softmax(tf.matmul(fc_out1, out_w1) + out_b1)

loss = -tf.reduce_mean(tf_Y*tf.log(tf.clip_by_value(pred, 1e-11, 1.0)))
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
y_pred = tf.arg_max(pred, 1)
bool_pred = tf.equal(tf.arg_max(tf_Y, 1), y_pred)
accuracy = tf.reduce_mean(tf.cast(bool_pred, tf.float32))

Tensor("Relu:0", shape=(?, 5, 7, 32), dtype=float32)
Tensor("Relu_1:0", shape=(?, 5, 7, 64), dtype=float32)
Tensor("Relu_2:0", shape=(?, 5, 7, 128), dtype=float32)
Tensor("batchnorm/add_1:0", shape=(?, 5, 7, 128), dtype=float32)


In [422]:
import random
def batch_get(X,Y,n_examples, batch_size):
    for batch_i in range(n_examples // batch_size):
        start = batch_i*batch_size
        end = start + batch_size
        batch_xs = X[start:end]
        batch_ys = Y[start:end]
        yield batch_xs, batch_ys # 生成每一个batch
batch_size = 20
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(2000):
        for batch_xs, batch_ys in batch_get(X_tr, Y_tr, Y_tr.shape[0], batch_size):
            sess.run(train_step, feed_dict={tf_X:batch_xs, tf_Y:batch_ys})
        if epoch % 50 == 0:
            res = sess.run(accuracy, feed_dict={tf_X:X_tr, tf_Y:Y_tr})
            lo=sess.run(loss,feed_dict={tf_X:X_tr,tf_Y:Y_tr})
            test_res = sess.run(accuracy, feed_dict={tf_X:X_te, tf_Y:Y_te})
            print ('%d, %g, %g' % (epoch, lo, test_res))

0, 0.5715, 0.786536
50, 0.190565, 0.890129
100, 0.180035, 0.893268
150, 0.176448, 0.897454
200, 0.172999, 0.896407
250, 0.17235, 0.893268


KeyboardInterrupt: 